导入数据

In [393]:
import pandas as pd
import numpy as np
import os

导入原始数据

In [394]:
data_path = "实际案例 - 零件特征及加工时间.xlsx"
order_data = pd.read_excel(data_path,sheet_name="订单信息")
workshop_data = pd.read_excel(data_path,sheet_name="车间信息")
workshop_data.head()

,车间设备总览表,Unnamed: 1,Unnamed: 2
0,序号,ID,描述
1,1,C1,1号数控机床
2,2,C2,2号数控机床
3,3,C3,3号数控机床
4,4,C4,4号数控机床


首先对车间信息表进行处理，即将列名称换成序号、ID以及描述

In [395]:
row_name = workshop_data.loc[0,:].tolist()
workshop_data = workshop_data.drop(axis=0,index=0)
workshop_data.columns = row_name
workshop_data.set_index('ID',inplace=True)
workshop_data.head()


,序号,描述
ID,,
C1,1,1号数控机床
C2,2,2号数控机床
C3,3,3号数控机床
C4,4,4号数控机床
L1,5,拉床


接下来对订单信息进行修改，将订单信息分割成加工时间以及加工机器两个信息表

In [396]:
workshop_order_row_name = ["零件","数量","特征","工序"]
workshop_order_row_name = workshop_order_row_name + [s.strip() for s in order_data.iloc[0,4:].tolist()]
order_data = order_data.drop(axis=0,index=0)
order_data.columns = workshop_order_row_name
order_data.set_index('零件',inplace=True)
order_data.head()

,数量,特征,工序,C1,C2,C3,C4,L1,PM1,NYM1,...,WX1,LX1,LX2,LX3,Z1,Z2,DHH1,XQG1,JC1,QG1
零件,,,,,,,,,,,,,,,,,,,,,
导向套,4.0,左端面,1.0,10,10,10,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
导向套,NaN,42圆柱面,2.0,10,10,10,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
导向套,NaN,Ra1.6环面,3.0,10,10,10,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
导向套,NaN,60圆柱面,4.0,40,40,40,40,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
导向套,NaN,48圆柱面,5.0,19,19,19,19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


得到零件的名字

In [397]:
def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

In [398]:
parts_name = order_data.index.dropna().to_list()
parts_name = remove_duplicates(parts_name)
parts_name

['导向套',
 '导向头',
 '反力支撑柱',
 '连接法兰',
 '压杆',
 '压头',
 '压装接头',
 '止转套',
 '导套安装座',
 '导向杆',
 '浮动连接座',
 '过渡杆',
 '卡簧压头',
 '锁紧压杆',
 '锁紧压块',
 '预压杆',
 '拉紧气缸接头',
 '止转销',
 '带锁气缸接头',
 '解锁推杆安装座',
 '抓具连接柱']

得到零件加工的数量

In [399]:
parts_process_num = np.array([10,18,7,10,9,11,9,7,8,9,9,8,4,5,4,7,5,7,7,8,10])
parts_num = order_data.loc[:,"数量"].dropna().to_numpy()
np.sum(parts_num*parts_process_num)

476.0

In [400]:
machines = np.zeros((21,18),dtype='object')
times = np.zeros((21,18),dtype='object')
sum_num = int(0)
for i in range(len(parts_name)):
    parts_data = order_data.loc[parts_name[i],:]
    parts_data.set_index("工序",inplace=True)
    num_process = order_data.loc[parts_name[i],"工序"].tolist()
    for j in range(len(num_process)):
        process_parts_data = parts_data.loc[num_process[j],"C1":"QG1"].dropna()
        machines[i,j] = process_parts_data.index.to_numpy()
        times[i,j] = process_parts_data.values





In [401]:
workshop_data.head()
mapping_dict = dict(zip(workshop_data.index.tolist(),(workshop_data.loc[:,"序号"]-1).to_list()))
mapping_dict

{'C1': 0,
 'C2': 1,
 'C3': 2,
 'C4': 3,
 'L1': 4,
 'PM1': 5,
 'NYM1': 6,
 'WYM1': 7,
 'RCL1': 8,
 'RCL2': 9,
 'WX1': 10,
 'LX1': 11,
 'LX2': 12,
 'LX3': 13,
 'Z1': 14,
 'Z2': 15,
 'DHH1': 16,
 'XQG1': 17,
 'JC1': 18,
 'QG1': 19}

In [402]:
def map_function(element):
    return mapping_dict.get(element, element)

mapped_machines = np.vectorize(map_function)
for i in range(len(parts_name)):
    for j in range(18):
        machines[i,j] = mapped_machines(machines[i,j])
machines


array([[array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]),
        array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]),
        array([ 0,  1,  2,  3, 11, 12, 13]),
        array([ 0,  1,  2,  3, 11, 12, 13]), array([4]), array([4]),
        array(0), array(0), array(0), array(0), array(0), array(0),
        array(0), array(0)],
       [array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]),
        array([0, 1, 2, 3]), array([0, 1, 2, 3]),
        array([ 0,  1,  2,  3, 11, 12, 14, 15]),
        array([ 0,  1,  2,  3, 11, 12, 14, 15]),
        array([ 0,  1,  2,  3, 13]), array([ 0,  1,  2,  3, 13]),
        array([11, 12, 13]), array([11, 12, 13]),
        array([11, 12, 13, 14, 15]), array([11, 12, 13]),
        array([ 4, 11, 12, 13]), array([ 4, 11, 12, 13, 14, 15]),
        array([11, 12, 13]), array([11, 12, 13, 14, 15]), array([17])],
       [array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]),
        array([11, 12, 13]), array([11, 12, 1

In [403]:
total_machines = np.zeros((56,18),dtype='object')
total_times = np.zeros((56,18),dtype='object')
sum_num = int(0)
for i in range(len(parts_num)):
    for j in range(int(parts_num[i])):
        total_machines[int(sum_num+j)] = machines[i]
        total_times[int(sum_num+j)] = times[i]
    sum_num += parts_num[i]


total_machines = total_machines.reshape(-1)
total_times = total_times.reshape(-1)
total_machines.squeeze()
total_times.squeeze()

array([array([10, 10, 10, 10], dtype=object),
       array([10, 10, 10, 10], dtype=object),
       array([10, 10, 10, 10], dtype=object), ..., 0, 0, 0], dtype=object)

In [404]:
delete_index = []
for i in range(1008):
    if np.any(total_machines[i]) == False:
        delete_index.append(i)

delete_index = np.array(delete_index,dtype=np.int16)
total_machines = np.delete(total_machines,delete_index)
total_times = np.delete(total_times,delete_index)

(476,)


(476,)